In [252]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/arabic-sarcasm-detection/Mini Projet8.docx
/kaggle/input/arabic-sarcasm-detection/Sceintfic_Report_Model.pdf
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/LICENSE
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/README.md
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv
/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv


In [233]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, GaussianNB, ComplementNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [234]:
df = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/training_data.csv')
print(df)
df_test = pd.read_csv('/kaggle/input/arabic-sarcasm-detection/Dataset/ArSarcasm-v2-main/ArSarcasm-v2-main/ArSarcasm-v2/testing_data.csv')
print(df_test)

                                                   tweet  sarcasm sentiment  \
0      "د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...    False       NEU   
1      "مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...    False       NEU   
2      “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...     True       NEG   
3      "@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...     True       NEG   
4      "قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...    False       NEU   
...                                                  ...      ...       ...   
12543  هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...     True       NEG   
12544  "لا اله الا الله💜#أيفون_البروفيسور https://t.c...    False       NEU   
12545  "RT @turkyepost: #أردوغان : إذا كان المرتكب غي...    False       NEU   
12546  RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...    False       POS   
12547        النت يلعن شكله طول الوقت تمام لين بدا الحفل     True       NEG   

      dialect  
0         msa  
1         msa  
2  

In [235]:
print(df.describe())

                                                    tweet sarcasm sentiment  \
count                                               12548   12548     12548   
unique                                              12487       2         3   
top     "RT @ahlalsunna2: #سوريا :الجيش السوري الحر يس...   False       NEU   
freq                                                    3   10380      5747   

       dialect  
count    12548  
unique       5  
top        msa  
freq      8562  


In [236]:
df.head(5)

,tweet,sarcasm,sentiment,dialect
0,"""د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق ر...",False,NEU,msa
1,"""مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHb...",False,NEU,msa
2,“الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...,True,NEG,msa
3,"""@ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasaf...",True,NEG,gulf
4,"""قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حل...",False,NEU,msa


In [237]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12548 entries, 0 to 12547
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   tweet      12548 non-null  object
 1   sarcasm    12548 non-null  bool  
 2   sentiment  12548 non-null  object
 3   dialect    12548 non-null  object
dtypes: bool(1), object(3)
memory usage: 306.5+ KB


In [238]:
print(df.columns)

Index(['tweet', 'sarcasm', 'sentiment', 'dialect'], dtype='object')


In [239]:
df['tweet'] = df['tweet'].str.strip('"')
print(df['tweet'])

0        د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق رق...
1        مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHbkBNu
2        “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...
3        @ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasafa...
4        قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حلب...
                               ...                        
12543    هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...
12544    لا اله الا الله💜#أيفون_البروفيسور https://t.co...
12545    RT @turkyepost: #أردوغان : إذا كان المرتكب غير...
12546    RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...
12547          النت يلعن شكله طول الوقت تمام لين بدا الحفل
Name: tweet, Length: 12548, dtype: object


In [240]:
labels, texts = df["sarcasm"], df["tweet"]
print(labels)
print(texts)

0        False
1        False
2         True
3         True
4        False
         ...  
12543     True
12544    False
12545    False
12546    False
12547     True
Name: sarcasm, Length: 12548, dtype: bool
0        د. #محمود_العلايلي:أرى أن الفريق #أحمد_شفيق رق...
1        مع فيدرر يا آجا والكبار 😍 https://t.co/hrBeHbkBNu
2        “الداعون لمبدأ الاختلاط بين الجنسين؛ كالداعين ...
3        @ihe_94 @ya78m @amooo5 @badiajnikhar @Oukasafa...
4        قل شرق حلب ولا تقل حلب الشرقية ....وقل غرب حلب...
                               ...                        
12543    هلأ صاير انت يا #فلعوط بدك تعطي محاضرات ع تويت...
12544    لا اله الا الله💜#أيفون_البروفيسور https://t.co...
12545    RT @turkyepost: #أردوغان : إذا كان المرتكب غير...
12546    RT @Yousiif65: هاري بوتر؟👓🎩 https://t.co/959Oo...
12547          النت يلعن شكله طول الوقت تمام لين بدا الحفل
Name: tweet, Length: 12548, dtype: object


In [241]:
labels_test, texts_test = df_test["sarcasm"], df_test["tweet"]
print(labels_test)
print(texts_test)

0       False
1        True
2       False
3       False
4        True
        ...  
2995    False
2996    False
2997    False
2998    False
2999     True
Name: sarcasm, Length: 3000, dtype: bool
0       اخوي حانق يالغلا وشفيك معصب؟ عادي تراهم بشر يف...
1          اف مو متعوده عليهم سته https://t.co/8igFPx1i26
2                 اللهم اشفِ مرضانا ومرضى المسلمين . . ♥️
3                                    ابشركم طلقت السات 😘.
4       مؤشر خطير: ٩٠٪ من الشخصيات البرلمانية في الكوي...
                              ...                        
2995    تربكني الذكرى ليا مر طاريهوانسى البشر حولي وال...
2996                   وانا احسبهم الحين مايتركون حركاتهم
2997    #فههههههههههد_غشششششششاااااااام_الببببببصصصصصم...
2998    - لــو كــان الامـــر بيدي لأخفيت انهيار دموعي...
2999    َ نِسااء بابل كنّ يسْحرن الرّجل بالعيُون فقط ....
Name: tweet, Length: 3000, dtype: object


In [242]:
# One-hot encode categorical columns ('sentiment' and 'dialect')
encoder = OneHotEncoder(sparse_output=False)  # Use `sparse_output` instead of `sparse`
categorical_encoded = encoder.fit_transform(df[['sentiment', 'dialect']])
categorical_encoded_df = pd.DataFrame(categorical_encoded, columns=encoder.get_feature_names_out(['sentiment', 'dialect']))
X_encoded = pd.concat([df[['tweet']], categorical_encoded_df], axis=1)

In [243]:
vectorizer = TfidfVectorizer()
text_vectorized = vectorizer.fit_transform(X_encoded['tweet'])


In [244]:
X_final = pd.concat([pd.DataFrame(text_vectorized.toarray()), X_encoded.drop('tweet', axis=1)], axis=1)

In [245]:
X_train, X_test, y_train, y_test = train_test_split(X_final, labels, test_size=0.2, random_state=42)

In [248]:
from sklearn.ensemble import RandomForestClassifier
X_train.columns = X_train.columns.astype(str)
X_test.columns = X_test.columns.astype(str)
model = RandomForestClassifier()
model.fit(X_train, y_train)

RandomForestClassifier()

In [250]:

# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test)


In [251]:

# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
class_report = classification_report(y_test, y_pred)

#Affichage des résultats
print(f"Précision du modèle: {accuracy:.2%}")
print("Matrice de confusion:\n", conf_matrix)

Précision du modèle: 85.02%
Matrice de confusion:
 [[2049   42]
 [ 334   85]]
